# Equal Hamming Weights, the Unitary, and Hardware Transpilation and Compilation Workshop
## Introduction to Classiq and Qmod

The Classiq platform features a high-level quantum modeling language called Qmod. Qmod is compiled into concrete gate-level implementation using a powerful synthesis engine that optimizes and adapts the implementation to different target hardware/simulation environments.

In this workshop, we will learn how to write quantum models using Qmod. We will be using the Python embedding of Qmod, available as part of the Classiq Python SDK. We will learn basic concepts in the Qmod language, such as functions, operators, quantum variables, and quantum types. 

The [Classiq webpage](https://docs.classiq.io/latest/) covers these concepts more systematically and includes more examples. For specific function you can use the [reference manual](https://docs.classiq.io/latest/qmod-reference) tab.

For each exercise, find the solution at the bottom of the notebook.

### Preparations

Make sure you have a Python version of 3.9 through 3.12 installed.

Install Classiq’s Python SDK by following the instructions on this page: [Getting Started - Classiq](https://docs.classiq.io/latest/getting-started/).

### Designing a quantum program

The first step in quantum software development is designing your software and your algorithm. Classiq features a unique high-level modeling language called Qmod that naturally captures the core concepts of quantum algorithm design. There are two ways to design in Qmod:
* Directly, via the Classiq IDE using the Qmod native syntax
* With Classiq Python SDK package, that gives access to the Qmod language via Python

Once you finish designing your algorithm, you send it to the Classiq synthesis engine (compiler) to create a concrete quantum circuit implementation - a quantum program.

### Python Qmod Exercises - General Instructions

There must be a qfunc decorator to specify this is a quantum function.

Quantum functions manipulate quantum objects, which are represented using quantum variables. Every variable needs to be declared and initialized before it is used.

In order to synthesize and execute your Qmod code, you should:
1. Make sure you define a `main` function that calls functions you create.
2. Use `create_model` by running `qmod = create_model(main)` to construct a representation of your model.
3. You can synthesize the model (using `qprog = synthesize(qmod)`) to obtain an implementation - a quantum program.
4. You can then visualize the quantum program (`show(qprog)`) or execute it using `execute(qprog)`. See: [Execution - Classiq](https://docs.classiq.io/latest/getting-started/classiq_tutorial/execution_tutorial/). You can also execute it with the IDE after visualizing the circuit.


Through the following example, we will explain some basic Qmod principles.


## Example

Let's get started and understand Qmod through a concrete example.


Our task is to design a quantum algorithm that computes the arithmtic operation $y=x^2+1$ coherently for a quantum variable $|x\rangle$ that is a superposition of all the numbers between $0$ and $7$:
$\begin{equation}
|x\rangle = \frac{1}{\sqrt{8}}(|0\rangle+|1\rangle+\dots +|7\rangle.
\end{equation}$
The expected output is 

$\begin{equation}
|x\rangle |y\rangle = |x\rangle |x^2+1\rangle = \frac{1}{\sqrt{8}}\sum_{i=0}^{7}|i\rangle|i^2+1\rangle,
\end{equation}$
where $|x\rangle$ is entangled to $|y\rangle$.

Sounds complicated? The following code in Qmod with a few lines creates the desired algorithm with Classiq:

In [1]:
from classiq import Output, QNum, allocate, hadamard_transform, qfunc


@qfunc
def main(x: Output[QNum], y: Output[QNum]):

    allocate(4, x)
    hadamard_transform(x)  # creates a uniform superposition
    y |= x**2 + 1

The desired quantum program can be synthesized and viewed:


In [2]:
from classiq import create_model, show, synthesize

quantum_program = synthesize(create_model(main))
show(quantum_program)

ConnectTimeout: [Errno 60] Operation timed out

and also executed:


In [ ]:
from classiq import execute

results = execute(quantum_program).result()
results[0].value.parsed_counts

## Part 1 - Apply Unitary 

General guideline to this workshop -- uncomment the relevant exercise block when implementing it, and fill in the missing code.
We will look at two approaches to implement the use case.
The first is brute force, which works on any Unitary. Here, only transpilation is used. Many times, more optimizations are applied.
In the second approach, we will look at the logic of the permutation and implement it.


### Task 1 - Apply Unitary Operator

Here, you will need to define a quantum function that applies a given unitary operator to a set of qubits. The unitary operator is given as a matrix. The function should take a set of qubits and apply the unitary operator to them.
Please note the following points:

* In Qmod, there are three types of quantum variables: `QBit`, `QNum`, `QArray[QBit]` (or a `QArray` of any other quantum type). See [quantum variables](https://docs.classiq.io/latest/qmod-reference/language-reference/quantum-variables) and [quantum types](https://docs.classiq.io/latest/qmod-reference/language-reference/quantum-types).
* Ensure you allocate the qubits before applying the unitary operator. See [allocate](https://docs.classiq.io/latest/qmod-reference/api-reference/operations/?h=allocate#classiq.qmod.builtins.operations.allocate).
* The unitary operator is given as a matrix. You can use the `unitary` function to apply the unitary operator to the qubits. See [unitary](https://docs.classiq.io/latest/explore/functions/qmod_library_reference/qmod_core_library/unitary/unitary/?h=unitary).
* See also explanation on the [main](https://docs.classiq.io/latest/qmod-reference/language-reference/quantum-entry-point) and on quantum [functions](https://docs.classiq.io/latest/qmod-reference/language-reference/functions).

Use the following unitary matrix:


In [ ]:
UNITARY = [
    [
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ],
    [
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ],
    [
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ],
    [
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ],
    [
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ],
    [
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ],
    [
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ],
    [
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ],
    [
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ],
    [
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ],
    [
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ],
    [
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ],
    [
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ],
    [
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ],
    [
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ],
    [
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ],
    [
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ],
    [
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ],
    [
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ],
    [
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ],
    [
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ],
    [
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ],
    [
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ],
    [
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ],
    [
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ],
    [
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
    ],
    [
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
    ],
    [
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
    ],
    [
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
    ],
    [
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
    ],
    [
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
    ],
    [
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
    ],
]

In [ ]:
from classiq import *


@qfunc
def main(x: Output[QArray[QBit]]):
    # Use allocate to allocate 5 qubits (it's a (2**5=32)x32 matrix)
    # Use unitary to apply the UNITARY operator to the qubits
    pass  # delete the "pass" call and add your code here

In [ ]:
# Uncomment the following lines:

# qmod = create_model(main)
# qprog = synthesize(qmod)
# show(qprog)

# print("depth:", qprog.transpiled_circuit.depth, "width:", qprog.data.width)

### Task 2 - Explore Hardware-Aware Synthesis

In this exercise, you will explore the hardware-aware synthesis feature of Classiq. This feature allows you to synthesize quantum programs for specific quantum hardware. You can specify the backend service provider and the backend name in the preferences of the model.

Uncomment the commented lines in the code below and fill in a backend service provider and a backend name. Then, synthesize the quantum program and print the depth and width of the transpiled circuit. You can see the available backends in the Classiq IDE execution tab. You can look at it in the context of the previous exercise. 

Synthesize for the Azure Quantum backend service provider and the some IBM backend name, and then for IonQ and qpu.aria-1.


In [ ]:
# qmod = create_model(
#     entry_point=main,
# preferences=Preferences(
#     backend_service_provider="",
#     backend_name=""
# ),
# )

# synthesize
# show
# print depth and width

## Part 2 - Build the Quantum Logic
In the this approach, we will look at the logic of the permutation and implement it. This approach shows the power of the Qmod language in expressing quantum algorithms.

### Task 1 - Understand the Unitary
Observe the unitary matrix and understand the permutation it represents. 
* How many qubits are there? How many states are there?
* What does the permutation do? Which states does it operate on? Write down the permutation in terms of the states it operates on.
* Translate it to binary representation. What are the bits that change? What are the bits that remain the same? Wh

Write down your answers and plan your implementation. You may use the given hints for help.
The next task will guide you through a possible implementation. SPOILER ALERT! Proceed to the next task only when you're ready.

<details>
  <summary>Hint -- what does the unitary do?</summary>

It switches between the states: 3 and 11, 5 and 13, and 6 and 14 
 
3 = 00011, 11 = 01011,
5 = 00101, 13 = 01101,
6 = 00110, 14 = 01110,
 
 - 00011 -- 01011
 -  00101 -- 01101
  - 00110 -- 01110
</details>

<details>
  <summary>Hint -- observations</summary>
    
    1. What states does the permutation operator on? In all of them (on the left-hand side), there are two 1s in the three least significant bits.
    2. These three least significant bits are the same before and after the permutation.
    3. Therefore, we can see that the permutation is a bit flip of the 4th qubit, in specific situations.
    4. This flip happens only if the 5th qubit is 0, and if there are two 1s and one 0 in the 1st, 2nd and 3rd qubits together.

</details>

### Task 2 - The Quantum Building Blocks
Now that we have observed what this Unitary does, we will build the quantum building blocks for it. 
We need to flip the 4th qubit if (1) the 5th qubit is 0, and (2) if there are two 1s and one 0 in the 1st, 2nd and 3rd qubits together.

Let's plan the implementation:
1. We will need to allocate 5 qubits.
2. We will need to define relevant quantum variables on these qubits (MSB - 5th, target_flip - 4th, LSBs - 1st to 3rd).
3. We will use quantum control-flow to determine if the MSB qubit is 0.
4. We will use quantum arithmetics to determine if the LSBs have two 1s and one 0, and if so we'll flip the target_flip qubit.

Now, let's implement the logic in Qmod. 
We will first learn how to use the Classiq Platform to create the different building blocks.
We will tie all them together in the next task.

#### Quantum Variables (1 and 2)

Uncomment block and fill in the relevant missing code blocks

Define a Qstruct that represents the permutation. The Qstruct should have the following quantum variables: MSB, target_flip, and LSBs. These should be of the appropriate quantum types. See more on [QStruct](https://docs.classiq.io/latest/qmod-reference/language-reference/quantum-types/?h=qstruct#examples_3).
Also, define a main and allocate 5 qubits in it. When you execute this function, you can see the quantum variables in the quantum program results.

In [ ]:
# from classiq import *
#
# class PermutationStruct(QStruct):
#     # Define the quantum variables here
#
# @qfunc
# def main(result: Output[PermutationStruct]):
#     # allocate 5 qubits into the result variable
#
# qmod = create_model(main)
# qprog = synthesize(qmod)
# show(qprog)

#### Quantum Control-Flow (3)
Now, we will learn the use of control flow. In the next task, we will adapt it to our the Unitary application.
The `control` operator is the conditional operation, with the condition being that the control qubit is in the state |1>, then an operation is applied on the second qubit. This notion is generalized in QMOD to other control states, where the condition is specified as a comparison between a quantum numeric variable, symbolic expression, `QArray` for example the [multi-control gates](https://github.com/Classiq/classiq-library/blob/main/functions/function_usage_examples/mcx/mcx_example.ipynb) and `QBit` (for simple control operation) and a numeric value.
 **It is very similar to a classical `if` statement.** Quantum numeric variables are declared with class `QNum`.

See also [Numeric types](https://docs.classiq.io/latest/qmod-reference/language-reference/quantum-types).

In QMOD, this generalization is available as a native `control` function.

See also [control](https://docs.classiq.io/latest/qmod-reference/language-reference/statements/control), and follow the instructions:

1. Declare a `QNum` output argument using `Output[QNum]` and name it `x`.
2. Use the `|=` inplace allocation function to initialize it to `9`. Note that you don't need to specify the `QNum` attributes - size, sign, and fraction digits, as they are inferred at the point of initialization.
3. Execute the circuit and observe the results.
4. Declare another output argument of type `QBit` and perform a `control` such that under the condition that `x` is 9, the qubit is flipped. Execute the circuit and observe the results. Repeat for a different condition.

Example of control: `control(ctrl = x==1, operand = lambda: H(q))` where `q` is a qubit.
The `control` is applied only for states that `x==1` and apply a `H` gate on it.

**Fill in the missing parts and uncomment the last lines of the synthesis to see the circuit.**


In [ ]:
from classiq import *


# Your code here:
@qfunc
def main(x: Output[QNum], target: Output[QBit]) -> None:
    # Make an integer of the x
    # Allocate the target register to have 1 qubit
    # Use control that accept control(ctrl = , operand = )
    # In the operand, use the lambda function using the X gate
    # Your code here:
    pass


# qmod = create_model(main)
# qprog = synthesize(qmod)
# show(qprog)

#### Quantum arithmetics (4)
In this exercise, we will use quantum numeric variables and calculate expressions over them.

See details on the syntax of numeric types under [Quantum types](https://docs.classiq.io/latest/qmod-reference/language-reference/quantum-types).
See more on quantum expressions under [Numeric assignment](https://docs.classiq.io/latest/qmod-reference/language-reference/statements/numeric-assignment).

Create the following quantum programs:
1. Initialize variables `x=5`, `y=7` and computes `res = x + y`.

Guidance:
* Use the operator `|=` to perform out-of-place assignment of arithmetic expression.
* To initialize the variables, use the function `prepare_int`.

See an example:


In [ ]:
from classiq import *


@qfunc
def main(res: Output[QNum]):
    x = QNum("x")
    y = QNum("y")
    x |= 5
    y |= 7
    res |= x + y


qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

Create the following quantum programs:
1. Initialize variables `x=2`, `y=7` and computes `res = x + y`.
2. Initialize variables `x=2`, `y=7` and computes `res = x * y`.
3. Initialize variables `x=2`, `y=7`, `z=1` and computes `res = x * y - z`.

In [ ]:
from classiq import *


@qfunc
def main(res: Output[QNum]):
    pass  # your code here


qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

### Task 3 - Tie it all together

Uncomment block and fill in the relevant missing code blocks.


In [ ]:
# from classiq import *
#
# class PermutationStruct(QStruct):
#     pass
#     # Define the quantum variables here
#
# # Implement a quantum function that receives two variable: a qubit and a quantum-array.
# # This function will check if the quantum-array has two 1s and one 0, and if so, flip the qubit.
# @qfunc
# def set_target(target: QBit,  lsb: QArray[QBit, 3]):
#     # Sum up the values in the array and compare it to the needed value
#     # Flip the target qubit if the condition is met
#     pass # delete the "pass" call and add your code here
#
# @qfunc
# def unitary_permute(x: PermutationStruct):
#     # Use control to call the "set_target" function, if the MSB is 0
#     pass # delete the "pass" call and add your code here
#
# @qfunc
# def main(result: Output[QNum]):
#     pass
#     # allocate 5 qubits into the result variable
#
#     # call unitary_permute to apply unitary logic. Notice the implicit cast done here.
#
#
# qmod = create_model(main)
# qprog = synthesize(qmod)
# show(qprog)

### Task 4 - Make sure it works as expected

Now we want to make sure the permutation does as expected. For that, we will use the hadamard function to create a superposition of all the states and then copy the result to another variable. We will then apply the permutation and check if the quantum state is permuted as expected. For that, please see: [repeat](https://docs.classiq.io/latest/qmod-reference/api-reference/operations/?h=repeat#classiq.qmod.builtins.operations.repeat), [hadamard transform](https://docs.classiq.io/latest/explore/functions/qmod_library_reference/classiq_open_library/hadamard_transform/hadamard_transform/?h=hadam)

Uncomment block and fill in the relevant missing code blocks.

In [ ]:
# # Part 2 full solution test functionality
# from classiq import *
#
# # Use these three functions from the previous tasks
#
# # class PermutationStruct(QStruct):
# #
# # @qfunc
# # def set_target(target: QBit,  lsb: QArray[QBit, 3]):
# #
# # @qfunc
# # def unitary_permute(x: PermutationStruct):
#
# @qfunc
# def copy_var(x: QNum, y: QNum):
#     # copy the state of x to y - create an entanglement between the two states
#
# @qfunc
# def main(result: Output[QNum], reference: Output[QNum]):
#     # allocate 5 qubits into the result variable
#
#     # perform the reference trick:
#     # allocate 5 qubits into the reference variable
#     # hadamard_transform the result variable
#     # copy the result variable to the reference variable
#
#     # call unitary_permute to apply unitary logic. Notice the implicit cast done here.
#
#
# qmod = create_model(main)
# qprog = synthesize(qmod)
# show(qprog)

### Task 5 - Re-Explore Hardware-Aware Synthesis 
Explore the results of different properties of the quantum program with the hardware-aware synthesis feature.
You can also explore the power of the synthesis engine with different optimization parameters, such as "depth" and "width", and the trade-offs between them.

Uncomment block and fill in the relevant missing code blocks.

In [ ]:
# qmod = create_model(
#     entry_point=main,
#     preferences=Preferences(
#         backend_service_provider="", # fill in the backend service provider
#         backend_name="", # fill in the backend name
#     ),
#     constraints=Constraints(optimization_parameter="depth")  # "width"
# )
# qprog = synthesize(qmod)
# show(qprog)

## Solutions

In [ ]:
# Part 1

from classiq import *


@qfunc
def main(x: Output[QArray[QBit]]):
    allocate(5, x)
    unitary(UNITARY, x)


qmod = create_model(
    entry_point=main,
    preferences=Preferences(
        backend_service_provider="IonQ",  # Azure Quantum
        backend_name="qpu.aria-1",  # ankaa-2
    ),
)
qprog = synthesize(qmod)
show(qprog)

print("depth:", qprog.transpiled_circuit.depth, "width:", qprog.data.width)

In [ ]:
# Part 2 - tasks 2 - variables

from classiq import *


class PermutationStruct(QStruct):
    lsb: QArray[QBit, 3]
    target: QBit
    msb: QBit


@qfunc
def main(result: Output[PermutationStruct]):
    allocate(5, result)


qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

In [ ]:
# Part 2 - tasks 2 - control flow

from classiq import *


@qfunc
def main(x: Output[QNum], target: Output[QBit]) -> None:
    x |= 9
    allocate(1, target)
    control(x == 9, lambda: X(target))


model = create_model(main)
qprog = synthesize(model)
show(qprog)

In [ ]:
# Part 2 - tasks 2 - arithmetics

from classiq import *


@qfunc
def main(res: Output[QNum]) -> None:
    x = QNum("x")
    y = QNum("y")
    z = QNum("z")
    prepare_int(2, x)
    prepare_int(7, y)
    prepare_int(1, z)
    # res |= x + y
    # res |= x * y
    res |= x * y - z


model = create_model(main)
qprog = synthesize(model)
show(qprog)

In [ ]:
# Part 2 tie it all together full solution
from classiq import *


class PermutationStruct(QStruct):
    lsb: QArray[QBit, 3]
    target: QBit
    msb: QBit


# Generic solution for set_target:
# @qfunc(generative=True)
# def set_target(target: QBit,  lsb: QArray[QBit, 3]):
#     target ^= (sum([lsb[i] for i in range(lsb.size)]) == 2)


@qfunc
def set_target(target: QBit, lsb: QArray[QBit, 3]):
    target ^= lsb[0] + lsb[1] + lsb[2] == 2


@qfunc
def unitary_permute(x: PermutationStruct):
    control(x.msb == 0, lambda: set_target(x.target, x.lsb))


@qfunc
def main(result: Output[QNum]):
    allocate(5, result)

    # apply unitary logic
    unitary_permute(result)


qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

In [1]:
# Part 2 full solution test functionality
from classiq import *


class PermutationStruct(QStruct):
    lsb: QArray[QBit, 3]
    target: QBit
    msb: QBit


@qfunc
def copy_var(x: QNum, y: QNum):
    # Another solution: repeat(x.len, lambda i: CX(x[i], y[i]))
    y ^= x


@qfunc
def set_target(target: QBit, lsb: QArray[QBit, 3]):
    target ^= lsb[0] + lsb[1] + lsb[2] == 2


@qfunc
def unitary_permute(x: PermutationStruct):
    control(x.msb == 0, lambda: set_target(x.target, x.lsb))


@qfunc
def main(result: Output[QNum], reference: Output[QNum]):
    allocate(5, result)

    # reference trick
    allocate(5, reference)
    hadamard_transform(result)
    copy_var(result, reference)

    # apply unitary logic
    unitary_permute(result)


qmod = create_model(main)
write_qmod(
    qmod,
    "hamming_weights",
    decimal_precision=15,
)
qprog = synthesize(qmod)
show(qprog)

Quantum program link: https://platform.classiq.io/circuit/36qDunRQkhG45iIXW1CZvc2AfQQ


In [ ]:
# Part 2 - tasks 5 - hardware-aware synthesis

qmod = create_model(
    entry_point=main,
    preferences=Preferences(
        backend_service_provider="Azure Quantum",
        backend_name="rigetti.qpu.ankaa-2",
    ),
    constraints=Constraints(optimization_parameter="depth"),  # "width"
)
qprog = synthesize(qmod)
show(qprog)